In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datasets import Dataset
from transformers import T5ForConditionalGeneration,T5Tokenizer,Trainer,TrainingArguments



In [6]:
train_data=pd.read_csv("/content/samsum-train.csv")
val_data = pd.read_csv("/content/samsum-validation.csv")

In [7]:
train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [8]:
val_data.head()

,id,dialogue,summary
0,13817023,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...
1,13716628,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...
2,13829420,Jackie: Madison is pregnant\r\nJackie: but she...,Madison is pregnant but she doesn't want to ta...
3,13819648,Marla: <file_photo>\r\nMarla: look what I foun...,Marla found a pair of boxers under her bed.
4,13728448,Robert: Hey give me the address of this music ...,Robert wants Fred to send him the address of t...


In [9]:
train_data.shape,val_data.shape

((14732, 3), (818, 3))

In [10]:
train_data["dialogue"][1]

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [11]:
#  text = text.translate(str.maketrans("","",string.punctuation))
#   text = text.lower().split()
#   text = " ".join(wn.lemmatize(word) for word in text if word not in sw)

In [12]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("wordnet")

wn=WordNetLemmatizer()
sw = stopwords.words("english")

def pre_processing(text):
  if isinstance(text,str):
    text = re.sub(r"\r\n", " ", text)
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"https?://\S+"," ",text)
    text = text.strip().lower()
  else:
    text =""
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
train_data["dialogue"] = train_data["dialogue"].apply(pre_processing)
train_data["summary"] = train_data["summary"].apply(pre_processing)


val_data["dialogue"] = val_data["dialogue"].apply(pre_processing)
val_data["summary"] = val_data["summary"].apply(pre_processing)

In [14]:
train_data.head()

,id,dialogue,summary
0,13818513,amanda: i baked cookies. do you want some? je...,amanda baked cookies and will bring jerry some...
1,13728867,olivia: who are you voting for in this electio...,olivia and olivier are voting for liberals in ...
2,13681000,"tim: hi, what's up? kim: bad mood tbh, i was g...",kim may try the pomodoro technique recommended...
3,13730747,"edward: rachel, i think i'm in ove with bella....",edward thinks he is in love with bella. rachel...
4,13728094,sam: hey overheard rick say something sam: i ...,"sam is confused, because he overheard rick com..."


In [15]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
max(len(tokenizer.encode(word)) for word in train_data["summary"])

108

In [17]:
def tokenization(example):
  input = tokenizer(example["dialogue"],padding="max_length",truncation=True,max_length=512)
  target = tokenizer(example["summary"],padding="max_length",truncation=True,max_length=110)
  input["labels"]=target["input_ids"]
  return input

train_data = Dataset.from_pandas(train_data)
train_data = train_data.map(tokenization, batched=True)


# train_data.set_format(type="torch",columns=["input_ids","attention_mask","labels"])
val_data = Dataset.from_pandas(val_data)
val_data = val_data.map(tokenization,batched=True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [18]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [21]:
training_args = TrainingArguments(
    output_dir="./output",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_steps=300,
    weight_decay=0.01,
    eval_steps=50


)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Step,Training Loss
300,0.523000
600,0.526100
900,0.511700
1200,0.512000
1500,0.509700
1800,0.502300
2100,0.495300
2400,0.505900
2700,0.511000


Step,Training Loss
300,0.523000
600,0.526100
900,0.511700
1200,0.512000
1500,0.509700
1800,0.502300
2100,0.495300
2400,0.505900
2700,0.511000
3000,0.503300


TrainOutput(global_step=3684, training_loss=0.5080975761372155, metrics={'train_runtime': 2417.2502, 'train_samples_per_second': 24.378, 'train_steps_per_second': 1.524, 'total_flos': 7975421677142016.0, 'train_loss': 0.5080975761372155, 'epoch': 4.0})

In [28]:
trainer.save_model("./model")
tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/spiece.model',
 './tokenizer/added_tokens.json')

In [30]:
from transformers import pipeline,T5Tokenizer,T5ForConditionalGeneration

tokenizer_path ="/content/tokenizer"
model_path = "/content/model"
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

def summary(text):
  summary = summarizer(text,max_length=150,do_sample=False)
  return summary[0]

text = "Artificial Intelligence (AI) has been transforming various sectors, from healthcare to finance. With advancements in machine learning algorithms and the development of powerful computational resources, AI has become more capable and accessible. In healthcare, AI is being used to diagnose diseases more accurately and quickly, sometimes even outperforming human doctors in tasks like interpreting medical imaging. In the financial sector, AI helps in predicting market trends, automating trading, and enhancing customer service through chatbots and personalized recommendations.Despite the immense potential, there are challenges to be addressed, such as ensuring the ethical use of AI, mitigating biases in algorithms, and ensuring that AI does not replace jobs unnecessarily. As AI continues to evolve, it is crucial that developers, policymakers, and the public collaborate to ensure that AI benefits society as a whole, without compromising fairness, privacy, or employment opportunities."

print("summary :",summary(text))


Device set to use cuda:0


summary : {'summary_text': 'AI is being used to diagnose diseases more accurately and quickly, sometimes outperforming human doctors in tasks like interpreting medical imaging. in the financial sector, AI helps in predicting market trends, automating trading, and enhancing customer service through chatbots and personalized recommendations.'}


In [31]:
!zip -r model.zip /content/model
!zip -r tokenizer.zip /content/tokenizer


  adding: content/model/ (stored 0%)
  adding: content/model/generation_config.json (deflated 29%)
  adding: content/model/model.safetensors (deflated 10%)
  adding: content/model/training_args.bin (deflated 52%)
  adding: content/model/config.json (deflated 63%)
  adding: content/tokenizer/ (stored 0%)
  adding: content/tokenizer/spiece.model (deflated 48%)
  adding: content/tokenizer/tokenizer_config.json (deflated 94%)
  adding: content/tokenizer/added_tokens.json (deflated 83%)
  adding: content/tokenizer/special_tokens_map.json (deflated 85%)
